<a href="https://colab.research.google.com/github/yangyeji2020311166/ML/blob/main/%EC%9D%98%EB%AF%B8_%EC%97%B0%EA%B2%B0%EB%A7%9D_%EC%82%AC%EC%A0%84%EC%B6%94%EA%B0%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import re
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy 
!JAVA_HOME="C:\Program Files\Java\jdk-17\"

In [ ]:
f=open('/content/스마트폰 크롤링.txt','r',encoding='utf-8')
txt = f.readlines()
f.close()

txt2=[]          #중복 댓글 제거
for item in txt:
    if item not in txt2:
        txt2.append(item)


print(txt2)

In [ ]:
##대댓글 아이디 삭제하기
remove_id_list=[]
for i in range(len(txt2)):
  if '?@' in txt2[i]:
    cnt = 0
    for j in range(len(txt2[i])):
      if (txt2[i][j] == '?' or txt2[i][j] == ' '):
        if (txt2[i][j] == '?'):
          cnt+=1
      elif cnt>=2:
        break
    tmp = txt2[i].split(sep='?',maxsplit=cnt)
    remove_id_list.append(tmp[cnt])
  else:
    remove_id_list.append(txt2[i])

print(remove_id_list)

In [ ]:
!git clone https://github.com/lovit/customized_konlpy.git
!pip install customized_konlpy

In [ ]:
from ckonlpy.tag import Twitter

twitter=Twitter()
twitter.add_dictionary(['유튜버','카툭튀','인트로','잇섭','플래그십','불닭','불닭볶음탕','불닭탕','호불호','오뚜기','컵누들','새우탕','칼국수','콕콕콕','불닭볶음면','까르보불닭','광천김라면','까르보나라','참치마요','치즈볶이','민생라면',
                        '장인라면','미역국라면','왕뚜껑','틈새라면','콩고기','진짬뽕','맛짬뽕','네넴띤','국민비빔면','비빔불닭면','칼빔면','정비빔면','배홍동비빔면',
                        '제주해물맛라면','팔도라면','호빵맨컵라면','튀김우동','제주해녀라면','독도사랑라면','삼양라면','불닭게티','꼬꼬면','돈코츠라멘','배홍동'], 'Noun')


In [ ]:
from konlpy.tag import Okt
import konlpy
from collections import Counter

for i, document in enumerate(remove_id_list):
    clean_words = []
    for word in twitter.pos(document, stem=True): #어간 추출
        if word[1] in ['Noun']: #이부분에 원하는 품사 넣기
            clean_words.append(word[0])
    remove_id_list[i] = clean_words

print(remove_id_list)

In [ ]:
#불용어 제거
f = open('/content/스탑워드.txt', "rt", encoding="utf-8") 
lines = f.readlines()
stopwords = []
unique_Noun_words=[]
for line in lines:
    line = line.replace('\n', '')
    stopwords.append(line)
f.close()
print(stopwords)

In [ ]:
unique_Noun_words=[]
for i in range(len(remove_id_list)):#이차원 배열 전체에서 for 문 돌리기
  tmp=[]
  for j in range(len(remove_id_list[i])):#배열 안에서 일차원 배열의 길이가 서로 다 다르니깐 그 길이만큼 돌리면서
    if remove_id_list[i][j] not in stopwords:#불용어 있는지 검사 없으면 tmp에 임시저장
      tmp.append(remove_id_list[i][j])
  unique_Noun_words.append(tmp)#tmp에 임시저장한 리스트를 배열 전체 하나가 끝날때마다 검사
#for i in range(len(unique_Noun_words)):
  #print(unique_Noun_words[i])

In [ ]:
len(unique_Noun_words)

In [ ]:
!pip3 install apyori

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

te = TransactionEncoder()
te_result = te.fit(unique_Noun_words).transform(unique_Noun_words)
df2 = pd.DataFrame(te_result, columns = te.columns_)
df2

In [ ]:
column_list = df2.columns
word_length = len(column_list)

from tqdm import notebook
import time
# 각 단어쌍의 빈도수를 저장할 dictionary 생성
count_dict = {}

for doc_number in notebook.tqdm(range(len(df2)), desc='단어쌍 만들기 진행중'):
    tmp = df2.loc[doc_number]           # 현재 문서의 단어 출현 빈도 데이터를 가져온다.
    for i, word1 in enumerate(column_list):
        if tmp[word1]:              # 현재 문서에 첫번째 단어가 존재할 경우
            for j in range(i + 1, word_length):
                if tmp[column_list[j]]:              # 현재 문서에 두번째 단어가 존재할 경우
                    count_dict[column_list[i], column_list[j]] = count_dict.get((column_list[i], column_list[j]), 0) + max(tmp[word1], tmp[column_list[j]])

# count_list에 word1, word2, frequency 형태로 저장할 것이다.
count_list = []

for words in count_dict:
    count_list.append([words[0], words[1], count_dict[words]])

# 단어쌍 동시 출현 빈도를 DataFrame 형식으로 만든다.
df3 = pd.DataFrame(count_list, columns=["word1", "word2", "freq"])
df3 = df3.sort_values(by=['freq'], ascending=False)
df3 = df3.reset_index(drop=True)
df3

In [ ]:
#####불용어 제거 부분
del df2['되다']
del df2['이다']
del df2['쓰다']
del df2['보다']
del df2['하다']
#######

result = apriori(df2, min_support=0.01, use_colnames=True)

result['length'] = result['itemsets'].apply(lambda x: len(x))
result = result[(result['length']==2) & (result['support']>= 0.01)].sort_values(by='support',ascending=False)
result.head(20)

In [ ]:
#연관규칙
#원래 작성했던 부분
#from apyori import apriori
#result = (list(apriori(unique_Noun_words, min_support = 0.01)))
#df = pd.DataFrame(result)
#df['length'] = df['items'].apply(lambda x: len(x))
#df = df[(df['length']==2) & (df['support']>= 0.01)].sort_values(by='support',ascending=False)
#df.head(20)

In [ ]:
G = nx.Graph()
ar=(result['items']); G.add_edges_from(ar)

In [ ]:
pr = nx.pagerank(G)
nsize = np.array([v for v in pr.values()])
nsize = 2000 * (nsize - min(nsize)) / (max(nsize) - min(nsize))

In [ ]:
pos = nx.random_layout(G)

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
plt.figure(figsize=(16,12));plt.axis('off')
nx.draw_networkx(G, font_family='NanumBarunGothic',font_size=16,
                 pos=pos, node_color=list(pr.values()), node_size=nsize,
                 alpha=0.7, edge_color='.5',cmap=plt.cm.YlGn)